### Apriori算法
* 从大规模数据集中寻找物品间的隐含关系被称作关联分析或关联规则学习。
* Apriori优点是易编码实现，缺点是在大数据集上可能较慢。
* 关联分析有两种形式：频繁项集和关联规则。频繁项集是经常出现在一块的物品的集合，关联规则暗示两种物品之间可能存在很强的关系。

* 频繁的定义：支持度和置信度
* 支持度：数据集中包含该项集的记录所占的比例；置信度：针对关联规则来说，诸如{尿布} ➞ {葡萄酒}，这条规则的可信度被定义为“支持度({尿布, 葡萄酒})/支持度({尿布})”。

Apriori原理：
* 如果某个项集是频繁的，那么它的所有子集也是频繁的。这意味着如果{0,1}是频繁的，那么{0}、{1}也一定是频繁的。这个原理直观上并没有什么帮助，但是如果反过来看就有用了，也就是说如果一个项集是非频繁集，那么它的所有超集也是非频繁的。

### 使用Apriori算法来发现频繁集
#### 1.频繁项集：生成候选项集
* 算法思路：C1是大小为1的所有候选项集的集合。Apriori算法首先构建集合C1，然后扫描数据集来判断这些只有一个元素的项集是否满足最小支持度的要求。那些满足最低要求的项集构成集合L1。而L1中的元素相互组合构成C2，C2再进一步过滤变为L2。

In [1]:
def loadDataSet():
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:     # [item]以列表形式
                C1.append([item])
                
    C1.sort()
    return list(map(frozenset, C1)) # frozenset类型是不能修改的,不能使用set。
                                    # 因为之后必须要将这些集合作为字典键值使用

* python中字典的键是必须是可哈希的对象。
* python中的类型可以分为两类：可变类型和不可变类型。
* 可变类型：dict、list、set
* 不可变类型：int、long、float、complex、string、bool、tuple
* 其中可变类型不能作为字典的key，因为它们没有__hash__()方法。(tuple作为字典中的键也有限制)

In [2]:
# 函数用于从Ck生成Lk，另外会返回一个包含支持度值的字典以备后用
def scanD(D, Ck, minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if can not in ssCnt: ssCnt[can]=1  # python3去掉了.has_key()
                else: ssCnt[can] += 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]/numItems
        if support >= minSupport:
            retList.insert(0,key)   #在列表的首部插入任意新的集合。当然也不一定非要在首部插入，这只是为了让列表看起来有组织。
        supportData[key] = support  # 保留所有项集的支持度（若只保留符合要求的项集支持度，则直接缩进即可）
    return retList, supportData

In [3]:
# 导入数据集
dataSet = loadDataSet()
dataSet

[[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [4]:
# 构建第一个候选项集集合C1：
C1 = createC1(dataSet)
C1 #C1包含了每个frozenset中的单个物品项

[frozenset({1}),
 frozenset({2}),
 frozenset({3}),
 frozenset({4}),
 frozenset({5})]

In [5]:
# 构建集合表示的数据集D
D = list(map(set,dataSet))  # set(list)针对一维列表
D

[{1, 3, 4}, {2, 3, 5}, {1, 2, 3, 5}, {2, 5}]

In [6]:
# 去掉不满足最小支持度的项集
L1,suppData0 = scanD(D,C1,0.5)
L1

[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})]

In [7]:
suppData0

{frozenset({1}): 0.5,
 frozenset({3}): 0.75,
 frozenset({4}): 0.25,
 frozenset({2}): 0.75,
 frozenset({5}): 0.75}

#### 2.组织完整的Apriori算法

* 对于python中集合内元素是无序的这句话，更正确的理解应该是，作为使用set这个功能的开发者，不应该假定set中的元素存在顺序。

In [3]:
# 创建Ck:输入参数为频繁项集列表Lk与(最终返回的)项集元素个数k，输出为Ck
def aprioriGen(Lk, k):
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk): 
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1==L2:                        # 如果列表的前k-2个元素相同
                retList.append(Lk[i] | Lk[j]) # 取并集（集合自动排序）
    return retList

In [4]:
def apriori(dataSet, minSupport = 0.5):
    C1 = createC1(dataSet)
    D = list(map(set, dataSet))
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    # 直到当Lk为空时，程序返回L并退出。 
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)   # 扫描数据集，从Ck得到Lk
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData

In [10]:
L,suppData = apriori(dataSet)
L   # 最后一个为空的原因是：aprioriGen()中最后返回的retList为空

[[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})],
 [frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})],
 [frozenset({2, 3, 5})],
 []]

### 从频繁项集中挖掘关联规则
* 关联规则量化方法：可信度
* 一条规则P ➞ H的可信度定义为:support(P,H)/support(P). （从逻辑研究上来讲，箭头左边的集合称作前件，箭头右边的集合称为后件。） 

In [11]:
def generateRules(L, supportData, minConf=0.7):  
    bigRuleList = []
    for i in range(1, len(L)):   # i从1开始，表示只取项数大于等于2的项集
        for freqSet in L[i]:
            # 对每个频繁项集集合的频繁项,生成单项集合,注意使用了frozenset,因为之后要用[item]作为key获取支持度
            H1 = [frozenset([item]) for item in freqSet]    # 例如{0,1,2} —— {0},{1},{2}
            if (i > 1):                                     # 元素大于两个的频繁项，将频繁项中的元素进行组合成后件（规则）
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)  # 计算置信度
    return bigRuleList          # 最终返回规则列表

In [12]:
def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    prunedH = []               # 返回关联规则中的后件列表 
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq]  # 可信度(freq-conseq)->conseq
        if conf >= minConf: 
            print (freqSet-conseq,'-->',conseq,'conf:',conf)
            brl.append((freqSet-conseq, conseq, conf))          # 保存规则
            prunedH.append(conseq)    # 保存后件（规则），为后面准备。注：不满足规则的后件，在该元素基础下添加也不会满足
    return prunedH

* 问题：如果直接合并，那么对于项数大于等于3的项集的后件只能大于等于2,而没有1的情况,可能是由于项数为2时已经可以得出所有项数为1的后件？？
* 规则中没有类似这样的规则：{2，3}->{5}
* rulesFromCq为书中代码，缺少规则。修改后为rulesFromConseq

In [13]:
def rulesFromCq(freqSet, H, supportData, brl, minConf=0.7):
    m = len(H[0])
    if (len(freqSet) > (m + 1)):  # 频繁项元素数目>后件（候选规则）的元素数，一直执行。例如{1，2，3，4}当{1}-{2,3,4}以后此时H[0]长度为3
        # 如果直接合并，那么对于项数大于等于3的项集的后件只能大于等于2,而没有1的情况,可能是由于项数为2时已经可以得出所有项数为1的后件？？
        Hmp1 = aprioriGen(H, m+1) # 合并候选后件（规则），由Hm生成Hm+1。注：不满足规则的后件，在该元素基础下添加也不会满足
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)  # 返回规则后件（规则）列表
        if (len(Hmp1) > 1):       # 返回后件（规则）列表包含1个以上后件（规则），递归进一步组合这些规则
            # 如果满足最小可信度的候选关联规则数目大于1，那么递归,将项数+1，继续进行过滤,直到候选关联规则数目小于等于1或者freqSet数目<=m+1，例如{1,2,3}不能以{1,2,3}为后件
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)

In [18]:
def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    m = len(H[0])
    if (len(freqSet) > (m)):  
        # 先计算项数为1的后件，再组合后件
        Hmp1 = calcConf(freqSet, H, supportData, brl, minConf)
        Hmp1 = aprioriGen(Hmp1, m+1) 
        if (len(Hmp1) > 1):       
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)

In [18]:
L,suppData = apriori(dataSet,minSupport=0.5)

In [19]:
# 原先的书中的规则列表
rules = generateRules(L,suppData,minConf=0.5)
rules

frozenset({3}) --> frozenset({2}) conf: 0.6666666666666666
frozenset({2}) --> frozenset({3}) conf: 0.6666666666666666
frozenset({5}) --> frozenset({3}) conf: 0.6666666666666666
frozenset({3}) --> frozenset({5}) conf: 0.6666666666666666
frozenset({5}) --> frozenset({2}) conf: 1.0
frozenset({2}) --> frozenset({5}) conf: 1.0
frozenset({3}) --> frozenset({1}) conf: 0.6666666666666666
frozenset({1}) --> frozenset({3}) conf: 1.0
frozenset({5}) --> frozenset({2, 3}) conf: 0.6666666666666666
frozenset({3}) --> frozenset({2, 5}) conf: 0.6666666666666666
frozenset({2}) --> frozenset({3, 5}) conf: 0.6666666666666666


[(frozenset({3}), frozenset({2}), 0.6666666666666666),
 (frozenset({2}), frozenset({3}), 0.6666666666666666),
 (frozenset({5}), frozenset({3}), 0.6666666666666666),
 (frozenset({3}), frozenset({5}), 0.6666666666666666),
 (frozenset({5}), frozenset({2}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0),
 (frozenset({3}), frozenset({1}), 0.6666666666666666),
 (frozenset({1}), frozenset({3}), 1.0),
 (frozenset({5}), frozenset({2, 3}), 0.6666666666666666),
 (frozenset({3}), frozenset({2, 5}), 0.6666666666666666),
 (frozenset({2}), frozenset({3, 5}), 0.6666666666666666)]

In [20]:
# 现在的规则列表
rules = generateRules(L,suppData,minConf=0.5)

frozenset({3}) --> frozenset({2}) conf: 0.6666666666666666
frozenset({2}) --> frozenset({3}) conf: 0.6666666666666666
frozenset({5}) --> frozenset({3}) conf: 0.6666666666666666
frozenset({3}) --> frozenset({5}) conf: 0.6666666666666666
frozenset({5}) --> frozenset({2}) conf: 1.0
frozenset({2}) --> frozenset({5}) conf: 1.0
frozenset({3}) --> frozenset({1}) conf: 0.6666666666666666
frozenset({1}) --> frozenset({3}) conf: 1.0
frozenset({3, 5}) --> frozenset({2}) conf: 1.0
frozenset({2, 5}) --> frozenset({3}) conf: 0.6666666666666666
frozenset({2, 3}) --> frozenset({5}) conf: 1.0
frozenset({5}) --> frozenset({2, 3}) conf: 0.6666666666666666
frozenset({3}) --> frozenset({2, 5}) conf: 0.6666666666666666
frozenset({2}) --> frozenset({3, 5}) conf: 0.6666666666666666


### 例：美国国会投票
* 由于在使用外部API时可能会遇到错误，并且也不想让错误占用数据获取的时间，上述try-except模块调用是一种非常可行的做法。有错误就会执行except中的内容。使用sleep(1)程序休眠一秒钟，避免对网站的过度频繁访问（from time import sleep)。

### 例：发现毒蘑菇的相似特征
* 有时我们并不想寻找所有频繁项集，而只对包含某个特定元素项的项集感兴趣。
* 会寻找毒蘑菇中的一些公共特征，利用这些特征就能避免吃到那些有毒的蘑菇。
* UCI的机器学习数据集合中有一个关于肋形蘑菇的23种特征的数据集，每一个特征都包含一个标称数据值。
* 我们必须将这些标称值转化为一个集合，已经有人做好了这种转换。
* Roberto Bayardo对UCI蘑菇数据集进行了解析，将每个蘑菇样本转换成一个特征集合。其中，枚举了每个特征的所有可能值，如果某个样本包含特征，那么该特征对应的整数值被包含数据集中。（mushroom.dat)

* 第一个特征表示有毒或者可食用。如果某样本有毒，则值为2。如果可食用，则值为1。下一个特征是蘑菇伞的形状，有六种可能的值，分别用整数3-8来表示。

In [6]:
mushDatSet = [line.split() for line in open('mushroom.dat').readlines()]
len(mushDatSet)

8124

In [7]:
L,suppData = apriori(mushDatSet,minSupport=0.3)

* 在结果中可以搜索包含有毒特征值2的频繁项集：

In [16]:
for item in L[1]:
    if item.intersection('2'): print(item)  
# set的方法：intersection() 方法用于返回两个或更多集合中都包含的元素，即交集。
# 括号内的参数可以不是集合，可以是任意序列。

frozenset({'2', '28'})
frozenset({'53', '2'})
frozenset({'2', '23'})
frozenset({'2', '34'})
frozenset({'2', '36'})
frozenset({'2', '59'})
frozenset({'63', '2'})
frozenset({'2', '67'})
frozenset({'76', '2'})
frozenset({'2', '85'})
frozenset({'2', '86'})
frozenset({'90', '2'})
frozenset({'2', '93'})
frozenset({'2', '39'})


* 也可以对更大的项集来重复上述过程:

In [18]:
for item in L[3]:
    if item.intersection('2'): print(item)

frozenset({'28', '59', '63', '2'})
frozenset({'28', '63', '2', '85'})
frozenset({'90', '28', '2', '39'})
frozenset({'90', '34', '28', '2'})
frozenset({'90', '28', '59', '2'})
frozenset({'90', '28', '2', '85'})
frozenset({'90', '28', '2', '86'})
frozenset({'34', '28', '59', '2'})
frozenset({'34', '28', '2', '85'})
frozenset({'34', '28', '2', '86'})
frozenset({'34', '28', '2', '39'})
frozenset({'28', '59', '2', '85'})
frozenset({'28', '59', '2', '86'})
frozenset({'28', '2', '86', '85'})
frozenset({'28', '59', '2', '39'})
frozenset({'28', '2', '39', '85'})
frozenset({'28', '2', '39', '86'})
frozenset({'34', '53', '2', '85'})
frozenset({'34', '53', '86', '2'})
frozenset({'34', '53', '39', '2'})
frozenset({'34', '28', '53', '2'})
frozenset({'53', '86', '2', '85'})
frozenset({'28', '53', '86', '2'})
frozenset({'90', '28', '53', '2'})
frozenset({'90', '53', '39', '2'})
frozenset({'90', '34', '53', '2'})
frozenset({'90', '53', '2', '85'})
frozenset({'90', '53', '86', '2'})
frozenset({'53', '39

* 接下来你需要观察一下这些特征，以便知道了解野蘑菇的那些方面。如果看到其中任何一个特征，那么这些蘑菇就不要吃了。

### 总结
* Apriori算法使用Apriori原理来减少在数据库上进行检查的集合的数目。
* Apriori原理是说如果一个元素项是不频繁的，那么那些包含该元素的超集也是不频繁的。Apriori算法从单元素项集开始，通过组合满足最小支持度要求的项集来形成更大的集合。支持度用来度量一个集合在原始数据中出现的频率。
* 每次增加频繁项集的大小，Apriori算法都会重新扫描整个数据集。当数据集很大时，这会显著降低频繁项集发现的速度。
* FP-growth算法：和Apriori算法相比，该算法只需要对数据库进行两次遍历，能够显著加快发现繁项集的速度。